In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix
import pickle
import os


from dataset_fr import datas
from torch.utils.data import DataLoader
import numpy as np

In [2]:
base_path=os.path.join('lfw')

In [3]:
fr_test=torch.load('face_recognition.pt')

In [4]:
inputFile = 'total.data'
fd = open(inputFile, 'rb')
all_images = pickle.load(fd)

In [5]:
faces=datas(all_images,base_path)
train_set, test_set = torch.utils.data.random_split(faces, [int(0.8*len(faces)), len(faces)-int(0.8*len(faces))])

In [6]:
dataloader=DataLoader(test_set)

In [7]:
y_compare=[]
with torch.no_grad():
    for batch in dataloader:
        img1, img2, y_target=batch
        y_pred=fr_test(img1.cuda(),img2.cuda())
        y_compare.append([y_pred.item(),y_target.item()])

In [8]:
y_np=np.round(y_compare,2).squeeze().astype(int)

In [9]:
print(confusion_matrix(y_np[:,1], y_np[:,0]))

[[4059    7]
 [ 220 3782]]


In [10]:
target_names = ['matched', 'not_matched']
print(classification_report(y_np[:,1], y_np[:,0], target_names=target_names))

              precision    recall  f1-score   support

     matched       0.95      1.00      0.97      4066
 not_matched       1.00      0.95      0.97      4002

    accuracy                           0.97      8068
   macro avg       0.97      0.97      0.97      8068
weighted avg       0.97      0.97      0.97      8068

